In [ ]:
import numpy as np
import pandas as pd
from scipy.special import logit, expit
from scipy import stats

# From getting to pre-processing data
In questa notebook affronteremo alcuni elementi che fanno parte di un progetto orientato al Machine Learning:
1. Ottenere i dati (facilitato)
2. Visualizzare i dati ottenuti
3. Trasformare/ manipolare i dati per fornire un input agli algoritmi di ML

A fine didattici/progetti personali, le fonti per ottenere dataset open sono molteplici:
- Kaggle (https://www.kaggle.com/)
- Machine Learning repository @ UC Irvine (https://archive.ics.uci.edu/ml/index.php)
- Reddit: subreddit dataset (https://www.reddit.com/r/datasets/)

Per questa prima esperienza utilizziamo un dataset ottenuto da Kaggle che affronta il problema dall **churn prediction**. Possiamo sintetizzare il problema in questo modo: *dato un insieme di clienti che hanno sottoscritto un servizio e caratterizzati/descritti da un insieme di proprieta', identificare un sottoinsieme di client che molto probabilmente abbondonera' in un dato momento nel futuro (di solito a breve termine) il servizio*. Il problema e' piuttosto generico in quanto riguarda in sostanza la risoluzione di un contratto.

<p style="border: 3px solid crimson;box-shadow: 5px 6px indianRed;padding:20px;">Formalmente ogni cliente $c$ e' descritto da un vettore $\mathbf{c}=<c_1,\ldots,c_n>$ t.c. $c_i\in S_i$, dove $S_i$ e' un insieme, denota la proprieta' $i$-esima che caratterizza l'utente $c$. Per predirre qualora un generico utente con determinate proprieta' al tempo $t$, al tempo $t+\Delta t$ abbandoni il servizio, possiamo **imparare** (learning) in modo automatico (machine) una funzione $f:S_1\times\ldots\times S_n\rightarrow \{0,1\}$, dove $0$ indica la permanenza nel servizio, viceversa $1$ indica l'abbandono del servizio.</p>

Nel nostro caso affrontiamo il problema di churn prediction in ambito bancario: credit card churn prediction. 

Il dataset e' disponibile all'indirizzo https://www.kaggle.com/sakshigoyal7/credit-card-customers e riporta la seguente descrizione:
```
A manager at the bank is disturbed with more and more customers leaving their credit card services. They would really appreciate if one could predict for them who is gonna get churned so they can proactively go to the customer to provide them better services and turn customers' decisions in the opposite direction.
This dataset consists of 10,000 customers mentioning their age, salary, marital_status, credit card limit, credit card category, etc. There are nearly 18 features.
We have only 16.07% of customers who have churned. Thus, it's a bit difficult to train our model to predict churning customers.
```

### Definire l'obiettivo
Lo sviluppo di un progetto che utilizzi ML solitamente e' orientato ad un **obiettivo** che non riguarda propriamente il ML, ma che utilizza le metodologie ML per **raggiungere** un'obiettivo. L'obiettivo determina:
- come modello il problema
    - supervisionato/non supervisionato, classificazione/regressione, batch/online, univariata/multivariata
- quali modelli/algoritmi adottare
    - fortemente dipendente dal punto precedente
- quali misure di performance utilizzare nella valutazione del modello

Nel nostro caso (credi card churn prediction) l'obiettivo e'molto specifico e ben definito, e la precedente trattazione ci porta a modellare il problema come un problema _supervisionato_ di _classificazione_ binaria in modalita' batch dato che il dataset e' facilmente caricabile in memoria.

## Getting and loading the data

Per un'analisi preliminare del dataset e caricarlo in memoria utilizziamo il modulo **pandas**

In [ ]:
raw_dataset = pd.read_csv('BankChurners.csv')

_Commento_: Potevo avere un'anteprima del file utilizzando anche un foglio di calcolo, dato che il formato e' CSV. E' utile verificare se la prima riga del file sia l'intestazione o header, e quindi contenere il nome delle colonne. Per verificare la presenza dello header posso utilizzare il comando head della shell

Per avere una prima intuizione sul tipo di dati messi a disposizione, possiamo visualizzare le prime 5 righe del dataset utilizzando il metodo **head**

In [ ]:
raw_dataset.head()

Ci sono 23 colonne.

Per avere una piu'completa descrizione della struttura del dataset utilizziamo i metodi **info** oppure **describe**. IL primo metodo e' piu'orientato alla rappresentazione del dato, mentre il secondo metodo riporta le statistiche riguardanti le varie colonne, qualora siano di tipo numerico.

In [ ]:
raw_dataset.info()

Confermiamo che abbiamo dati riguardanti 10127 clienti e in questo caso non abbiamo dati mancanti (Non-null count). I tipi di dati sono sia numerici sia categorici (Dtype).

In [ ]:
raw_dataset.describe()

Per avere anche una prima impressione sulle colonne di tipo categorico/testuale possiamo utilizzare il metodo **value_counts**.

🤡 Su diverse microblogging platforms trovare numerosi thread sulla controintuivita' del nome di questo metodo. SPOILER: la probabilita' di scriverlo correttamente al primo colpo e' prossima allo zero 🤡.

In [ ]:
raw_dataset['Gender'].value_counts()

Infine otteniamo il nome delle varie colonne in modo inferire - qualora non sia specificato in qualche documento - il significato della colonna. Tramite l'attributo **columns** otteniamo questa informazione

In [ ]:
raw_dataset.columns

| Nome colonna | Significato|
| --- | --- |
|CLIENTNUM| ID del cliente|
|Attrition_Flag| Indica se il cliente ha abbandonato (valore 1)|
|Customer_Age| Eta' del cliente in anni|
|Gender|Genere del cliente|
|Dependent_count| Numero di dipendenti|
|Education_Level| Livello di istruzione del cliente|
|Marital_Status| Stato civile|
|Income_Category| Livello di reddito in dollari|
|Card_Category| Tipo di carta di credito|
|Months_on_book| Numero di mesi di iscrizione al servizio|
|Total_Relationship_Count| Numero di prodotti in possesso del cliente|
|Months_Inactive_12_mon| Numero di mesi di inattivita' negli ultimi 12 mesi|
|Contacts_Count_12_mon| Numero di contatti negli ultimi 12 mesi|
|Credit_Limit| Limite di credito sulla carta|
|Total_Revolving_Bal| Bilancio revolving|
|Avg_Open_To_Buy|
|Total_Amt_Chng_Q4_Q1| Differenza del totale delle transazioni tra Q4 e Q1|
|Total_Trans_Amt| Totale delle transazioni su ultimi 12 mesi|
|Total_Trans_Ct| Numero di transazioni negli ultimi 12 mesi|
|Total_Ct_Chng_Q4_Q1| Differenza del numero di transazioni in Q4 rispetto a Q1|
|Avg_Utilization_Ratio| ?|
|Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1| Da eliminare|
|Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2| Da eliminare|

Un'analisi preliminare delle caratteristiche del dataset puo' avvenire in modo qualitativo attraverso la visualizzazione della distribuzione delle proprieta'. 

Utilizziamo il modulo **matplotlib** per la visualizzazione nel corso di questo laboratorio.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In pandas il metodo **hist** permette una rapida visualizzazione della distribuzione delle proprieta' numeriche attraverso istogrammi.

In [ ]:
# Prima di visualizzare le distribuzioni, rimuoviamo le colonne che iniziano con Naive Bayes e la colonna con il codice cliente
raw_dataset.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2','CLIENTNUM'],
                 axis=1,
                 inplace=True)

In [ ]:
raw_dataset.hist(figsize=(22,9))

Dal momento che il dataset non e' troppo grande, possiamo valutare il coefficiente di correlazione di Pearson mediante il metodo **corr**.

In [ ]:
matrice_correlazione = raw_dataset.corr()

In [ ]:
matrice_correlazione['Customer_Age'].sort_values(ascending=False)

Modificando il parametro _method_ posso cambiare il tipo di correlazione, verificando l'esistenza di una correlazione monotonica, non necessariamente lineare tra le proprieta'. Nel caso che vediamo utilizziamo il coefficiente di rank correlation di Spearman (Kendall rank correlation e' altra opzione).

In [ ]:
matrice_ranking_correlazione = raw_dataset.corr(method='spearman')

In [ ]:
matrice_ranking_correlazione['Customer_Age'].sort_values(ascending=False)

Ma il risultato non cambia :-)

### Visualizing the "correlation"
Un'analisi qualitativa della correlazione e' effettuabile attraverso la visualizzazione delle distribuzioni congiunte - a coppie - delle propriet' di tipo numerico.

In [ ]:
from pandas.plotting import scatter_matrix

Scegliamo un sottoinsieme di caratteristiche che hanno un diverso livello di correlazione con l'eta' del cliente.

In [ ]:
colonne_interesse = ['Customer_Age', 'Months_on_book', 'Credit_Limit', 'Total_Trans_Amt']

In [ ]:
scatter_matrix(raw_dataset[colonne_interesse], figsize=(22,9))

Per aumentare la leggibilita' e' possibile specificare la coppia di proprieta' di cui analizzare la distribuzione congiunta.

In [ ]:
raw_dataset.plot(kind='scatter', x = 'Customer_Age', y='Total_Trans_Ct', alpha=0.1)

In [ ]:
raw_dataset.plot(kind='scatter', x = 'Customer_Age', y='Months_on_book', alpha=0.1)

In questa fase di sperimentazione posso definire delle proprieta' che **combinano** le proprieta' iniziali.

Per esempio definiamo l'ammontare medio di una transazione engli ultimi 12 mesi.

In [ ]:
raw_dataset['Avg_transaction'] = raw_dataset['Total_Trans_Amt']/raw_dataset['Total_Trans_Ct']

..e ne visualizziamo la distribuzione.

In [ ]:
raw_dataset['Avg_transaction'].hist()

# Preparing the data for Machine Learning algorithms
In questa sezione prepariamo il nostro dataset per essere utilizzato da un modello di ML - nel nostro caso un classificatore binario.

**NB**: Prima di procedere, dobbiamo caricare un versione modificata del dataset orginale in cui sono state cancellate delle entry relative ad alcuni clienti e alcune collane. Tale modifica e' stata resa necessaria per introdurre alcuni step di data cleaning/data preparation relativi, che altrimenti non si sarebbero potuti applicare.

Carichiamo la nuova versione del dataset ed analizziamo quali feature sono state corrotte.

In [ ]:
credit_data = pd.read_csv('BankChurnersMissingData.csv')

In [ ]:
credit_data.info()

Le colonne 22 e 23 contengono molte entry con dati mancanti, mentre abbiamo dati mancanti sporadici nelle colonne Education Level, Months_Inactive_12_mon, Total_Revolving_Bal.

Molti algoritmi di ML non gestiscono i **dati mancanti**, di conseguenza una fase della preparazione dei dati e' la definizione di funzioni che gestiscono i dati mancanti. Le opzioni piu' utilizzate sono 3:
- rimuovo il sample che contiene le entry corrotte/mancanti => cancellazione della riga
- rimuovo l'intera feature/colonna contenente i dati mancanti/corrotti => rimozione della colonna
- imputo/inferisco i dati mancanti/corrotti mediante diverse strategie ben consolidate oppure originali

Prima di gestire i dati mancanti, vediamo come **identificare** alcuni sample che contengono un dato mancante.

In [ ]:
sample_missing_credit = credit_data[credit_data['Customer_Age'].isnull()]
sample_missing_credit

## Eliminating samples with missing values
Andiamo ad eliminare la riga relativa al cliente con un valore mancante nella colonna 'Total Revolving_Bal'. In questo caso non eseguiamo nessuna imputazione dal momento che togliamo un solo sample

In [ ]:
credit_data.dropna(subset=['Total_Revolving_Bal'],
                   inplace=True
                  )

In [ ]:
credit_data.info()

Ho eliminato solo una riga

## Eliminating features with missing values
Il dataset contiene due colonne che:
1) contengono molti dati mancanti
2) sono inutili al fine della classificazione in quanto contengono la predizione di un classificatore

In questo caso possiamo tranquillamente eliminare le colonne. In generale, tutttavia, l'elminazione di una feature deve seguire da una scelta ponderata sulle rimanenti opzioni per la gestione dei dati mancanti.

In [ ]:
credit_data.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                    'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
                   inplace=True,
                   axis=1
                  )

In [ ]:
credit_data.info()

## Imputing missing values
Abbiamo ancora una colonna che contiene 6 dati mancanti. In questo caso possiamo inferire il valore dei dati mancanti.

### Imputing missing values in Pandas
Gestiamo l'imputazione del dato mancante in Pandas, mediante alcune statistiche e il metodo **fillna**.Ci concentriamo sulla colonna 'Customer_Age'

In [ ]:
median_age = credit_data['Customer_Age'].median()
median_age
# credit_data['CustomerAge'].fillna(median_age, inplace=1)

### Imputing missing values in Scikit-Learn
Scikit-learn - SKL - offre una soluzione piu' completa per gestire l'imputazione dei dati mancanti.

La classe **SimpleImputer** fornisce una strategia base. I valori mancanti possono essere imputati da valori costanti oppure statistiche puntuali.

https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy='median')
imputer.fit(credit_data['Customer_Age'].values.reshape(-1, 1))
imputer.statistics_

Il metodo fit non esegue alcuna sostituzione ma esegue solamente la strategia utilizzata sul vettore o matrice che ricve in input.

In [ ]:
credit_data['Customer_Age'] = imputer.transform(credit_data['Customer_Age'].values.reshape(-1, 1))

Il metodo transform applica la strategia al vettore/matrice

In [ ]:
credit_data.info()

Abbiamo un ulteriore problema costituito dalla colonna 'Income_Category'.

Scikit-Learn API e' organizzato secondo una serie di principi che ne ispirano il design e l'architettura:

1) **Consistency**: ogni oggetto condivide un'interfaccia consistente e condivisa
    - **Estimator**: un qualsiasi oggetto che stima alcuni parametri. La stima e' eseguita dal metodo **fit** che richiede un dataset come parametro
    - **Transformers**: un estimator che trasforma un dataset. La trasformazione e' eseguita dal metodo **transform** che richiede il dataset da trasformare come parametro. Restuisce il dataset trasformato. Esiste un metodo **fit_trasform** che esegue entrambi i metodi ed e'ottimizzato.
    - **Predictors**: un oggetto estimator che predice. Un predictor ha il metodo **predict** che richiede un insieme di sample nuovi e restituisce le predizioni associate.
2) **Inspection**: tutti gli iperparametri sono accessibili mediante attributi cosi' come i parametri appresi. Si utilizza la notazione con underscore
3) **Nonproliferation of classes**: datasets sono Numpy arrays o matrici sparse Scipy. Non ci sono oggetti dataset definiti da utenti
4) **Composition**: i moduli/classi rilasciati sono componibili
5) **Sensible defaults**: si devono definire dei valori di default ragionevoli

In [ ]:
credit_data['Income_Category'].value_counts()

Una possibile soluzione e' quella di inferire i valori mancanti sulla base delle feature. In questo caso dobbiamo risolvere un problema di classificazione/regressione utilizzando le entry con i valori non mancanti come label. SKL mettere a disposizione la classe sperimentale **IterativeImputer** (https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer), tuttavia non e' possibile utilizzarlo con in valori categorici che abbiamo. 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
imputer_it = IterativeImputer(max_iter=10, random_state=0)
imputer_it.fit(credit_data[['Gender','Income_Category']].values)
IterativeImputer(random_state=0)
y = credit_data[['Gender','Income_Category']].values
imputer_it.transform(y)

In questo caso applichiamo una soluzione piu' 'naive' in cui i valori unknown vengono sostituiti in modo casuale, rispettando la distribuzione delle rimanenti livelli di reddito.

In [ ]:
count_ic = credit_data[credit_data['Income_Category'] != 'Unknown']['Income_Category'].value_counts()
v, f = count_ic.index, count_ic.values/count_ic.sum()
credit_data.loc[credit_data['Income_Category'] == 'Unknown','Income_Category'] = np.random.choice(v,size=1112,p=f)

In [ ]:
credit_data['Income_Category'].value_counts()

## Handling categorical data
Per ora abbiamo trattato principalmente dati numerici. Tuttavia, dal momento che molti algoritmi di apprendimento non trattano dati categori o testuali, dobbiamo codificare i dati categorici in dati numerici.

Un primo approccio per convertire un dato categorico in un dato categorico in un dato numerico e'utilizzare la classe **OrdinalEncoder**, la quale assegna ad ogni categoria un valore numerico distinto.

Applichiamo la classe alla colonna 'Card_Category'

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
ord_enc = OrdinalEncoder()
credit_data['Card_Category'] = ord_enc.fit_transform(credit_data['Card_Category'].values.reshape(-1, 1))

Le categorie identificate sono associate all'attributo **categories_**

In [ ]:
ord_enc.categories_

In [ ]:
credit_data.info()

Un problema di questo tipo di trasformazione e' l'introduzione di un ordinamento non presente - caso precedente, o il cambiamento dell'ordinamento. 

Trattiamo il secondo caso, trasformando la colonna categorica'Income_Category'. Nonostante sia una feature categorica, le fasce di reddito sono crescenti quindi esiste un ordinamento. Per rispettare questo ordinamento utilizziamo sempre la classe OrdinalEncoder, ma definiamo a priori la lista ordinata delle categorie.

In [ ]:
rank_cat = [['Less than $40K','$40K - $60K', '$60K - $80K', '$80K - $120K', '$120K +']]
ord_enc_income = OrdinalEncoder(categories=rank_cat)
credit_data['Income_Category_Num'] = ord_enc_income.fit_transform(credit_data['Income_Category'].values.reshape(-1, 1))

In [ ]:
credit_data[['Income_Category_Num','Income_Category']]

Per evitare l'introduzione di un ordimento fittizio, si crea una feature binaria per ogni categoria. Per ogni sample, viene posta a 1 la feature/colonna che corrisponde alla categoria originaria. Questo tipo di codifica viene detta **one-hot encoding** ed implementata dalla classe **OneHotEncoder** (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
edu_enc = OneHotEncoder()
edu_ohe = edu_enc.fit_transform(credit_data['Education_Level'].values.reshape(-1,1))

In [ ]:
edu_ohe

In [ ]:
edu_ohe.toarray().shape